In [1]:
from agents.document import Document
from agents.qa_agent import QAAgent
from models.embedder import ColQwen2Embedder, ColBertEmbedder
from models.database import QdrantManager
from config.settings import PROCESS_IMAGE_OUTPUT_FOLDER, DB_PATH, DB_PATH1
from langchain_openai import ChatOpenAI

import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:

class RAGTester:
    def __init__(self):
        # 初始化数据库和embedder
        self.text_db = QdrantManager(DB_PATH1, "text_collection")
        self.image_db = QdrantManager(DB_PATH, "image_collection")
        self.text_embedder = ColBertEmbedder()
        self.image_embedder = ColQwen2Embedder()
        self.qa_agent = QAAgent()
        self.current_doc = None

        
        os.environ.pop('http_proxy', None)
        os.environ.pop('https_proxy', None)
        os.environ.pop('HTTP_PROXY', None)
        os.environ.pop('HTTPS_PROXY', None)


    def process_pdf(self, pdf_path):
        """处理PDF文档"""
        try:
            # 创建Document对象并处理
            doc = Document(
                pdf_path,
                self.text_db,
                self.image_db,
                self.text_embedder,
                self.image_embedder,
                output_folder=PROCESS_IMAGE_OUTPUT_FOLDER,
            )
            
            # 处理文档
            success = doc.document_process()
            if not success:
                print(f"文档处理失败: {pdf_path}")
                return False
                
            # 更新当前文档
            if self.current_doc:
                try:
                    self.current_doc.delete()  # 删除旧文档
                except:
                    pass
                    
            self.current_doc = doc
            print(f"文档处理成功！")
            print(f"文档ID: {doc.doc_id}")
            print(f"文本块数量: {len(doc.text_blocks)}")
            print(f"图片数量: {len(doc.images)}")
            return True
            
        except Exception as e:
            print(f"处理文档时出错: {str(e)}")
            return False

    def ask_question(self, question):
        """提问并获取答案"""
        if not self.current_doc:
            print("请先处理PDF文档")
            return None
            
        # 构建对话历史
        history = [{"role": "user", "content": question}]
        
        # 获取答案
        answer = self.qa_agent.run(history, self.current_doc)
        return answer

    def cleanup(self):
        """清理当前文档"""
        if self.current_doc:
            try:
                self.current_doc.delete()
                print("已清理当前文档")
            except Exception as e:
                print(f"清理文档时出错: {str(e)}")




In [3]:

# 创建测试器实例
tester = RAGTester()

try:
    # 指定PDF路径
    pdf_path ="/root/MMRAG/colpali.pdf"
    
    # 处理PDF
    if tester.process_pdf(pdf_path):
        # 交互式问答
        print("\n开始问答测试 (输入'quit'退出)")
        question="i want to know what exactly colpali is？"
        answer = tester.ask_question(question)
        print("\n回答:", answer)

        
finally:
    # 清理资源
    tester.cleanup()

正在连接Qdrant数据库...
使用现有集合: text_collection
✅ Qdrant数据库连接成功
正在连接Qdrant数据库...
使用现有集合: image_collection
✅ Qdrant数据库连接成功
正在加载ColBERT模型...
✅ ColBERT模型 colbert-ir/colbertv2.0 加载完成
正在加载ColQwen2模型...


`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ ColQwen2模型加载完成
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	intent_recognition(intent_recognition)
	retrieve(retrieve)
	answer(answer)
	critic(critic)
	__end__([<p>__end__</p>]):::last
	__start__ --> intent_recognition;
	answer --> __end__;
	retrieve --> critic;
	intent_recognition -.-> retrieve;
	intent_recognition -.-> answer;
	critic -.-> retrieve;
	critic -.-> answer;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



CropBox missing from /Page, defaulting to MediaBox


content='\n\n你好！很高兴见到你。有什么我可以帮助你的吗？' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 6, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 12, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-R1-Distill-Qwen-32B', 'system_fingerprint': '', 'id': '01967b875aad4124bcce023fed838bad', 'finish_reason': None, 'logprobs': None} id='run-1763aa2d-5935-4d20-8487-23b28336855a-0' usage_metadata={'input_tokens': 6, 'output_tokens': 25, 'total_tokens': 31, 'input_token_details': {}, 'output_token_details': {'reasoning': 12}}
正在从 colpali.pdf 提取文本...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

KeyboardInterrupt: 